In [ ]:
%git clone https://github.com/Hotsnown/seminaire-bordeaux-2022.git seminaire &> /dev/null
%pip install nbautoeval &> /dev/null

from seminaire.evaluation.jour2.fonctions.project import exo_end_of_sentence_marker

# Mapping courts


In [ ]:
import os
import sys
sys.path.append('..')

import googlemaps

import csv
import json
import requests

import pandas as pd
import plotly.plotly as py

import matplotlib
%matplotlib inline

from config import settings

### Getting all courts from CAP API


Create a CSV file with all of the courts that exist in the CAP databse.
For each court, add the full jurisdiction name. This will be useful for locating it on a Google map.

In [ ]:
data_dir = "../%s/court_map" % settings.DATA_DIR
# Create data directory for the project if it doesn't already exist
if not os.path.exists(data_dir):
    os.mkdir(data_dir)
    print("Created %s" % data_dir)

In [ ]:
# Create courts csv if it doesn't already exist
all_courts_file = '%s/courts.csv' % data_dir
courts_fieldnames = ['court_id', 'court_slug', 'court_name', 'jurisdiction_name', 'jurisdiction_slug']
if not os.path.exists(all_courts_file):
    with open(all_courts_file, 'w+') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=courts_fieldnames)
        writer.writeheader()
    print("Created %s" % all_courts_file)
    
    # Get 100 courts at a time
    # For each court, get full jurisdiction name to help in Google Map searching
    courts_url = "%s/%s/courts/?format=json" % (settings.API_URL, settings.API_VERSION)
    res = requests.get(courts_url).json()
    while True:
        with open(all_courts_file, 'a+', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=courts_fieldnames)
            for court in res['results']:
                jurisdiction_url = court['jurisdiction_url']

                if not jurisdiction_url:
                    # if jurisdiction doesn't exist, skip
                    continue

                jur_res = requests.get(jurisdiction_url).json()

                writer.writerow({
                    'court_id': court['id'],
                    'court_slug': court['slug'],
                    'court_name': court['name'],
                    'jurisdiction_name': jur_res['name_long'],
                    'jurisdiction_slug': jur_res['slug']                    
                })

        if res['next']:
            # Get the next page of court results

            res = requests.get(res['next']).json()

        else:
            # That's it! No more results. 
            print("Done.")
            break

In [ ]:
df = pd.read_csv(all_courts_file)
df.head()

In [ ]:
# remove extra fields for graphing purposes
df.drop(columns=['court_id', 'court_slug', 'jurisdiction_slug'], inplace=True)
df.head()

Graph court numbers per jurisdiction


In [ ]:
aggregated_df = df.groupby(['jurisdiction_name']).agg(['count'])
aggregated_df.plot(kind='bar', legend=False, figsize=(50, 20), fontsize=30)